In [1]:
from deepface import DeepFace
#liveness
import tensorflow as tf

In [2]:
import cv2, numpy, datetime
import xlwings as xw
import numpy as np

In [3]:
import sys
print(sys.executable)


C:\Users\renji\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe


In [4]:
import os
import random
import mediapipe as mp

In [5]:
model = 'liveness.model'
model = tf.keras.models.load_model(model)

In [6]:
results = DeepFace.find(img_path='Renjith Anil.jpg', db_path = './ImagesAttendance',enforce_detection = False, model_name = 'VGG-Face')

24-11-07 01:48:43 - Searching Renjith Anil.jpg in 10 length datastore
24-11-07 01:48:47 - find function duration 4.00351357460022 seconds


In [7]:
results[0]['identity'][0]

'./ImagesAttendance\\Renjith Anil\\Renjith Anil.jpg'

In [8]:
results

[                                            identity  \
 0   ./ImagesAttendance\Renjith Anil\Renjith Anil.jpg   
 1  ./ImagesAttendance\Renjith Anil\Renjith Anil4.jpg   
 2  ./ImagesAttendance\Renjith Anil\Renjith Anil6.jpg   
 3  ./ImagesAttendance\Renjith Anil\Renjith Anil1.jpg   
 4  ./ImagesAttendance\Renjith Anil\Renjith Anil2.jpg   
 5  ./ImagesAttendance\Renjith Anil\Renjith Anil5.jpg   
 
                                        hash  target_x  target_y  target_w  \
 0  7c8ccbc0ae9af05fe49da4ad24bfd1bd471b244f         0         0       143   
 1  964f5e403a792adf272b5dfec532b4358fec4d30         0         0       863   
 2  c5c5875e209b86290b6395060b54d22e791d666d         0         0       863   
 3  83339303d2150b52479522db88cdb8ee053e4e0b         0         0       863   
 4  b63462bf488b1abb56142c90e2e0a4e6dbe3ef79         0         0       863   
 5  f98538a200f8c3b20d663b6396d95a63c68a39fa         0         0       863   
 
    target_h  source_x  source_y  source_w  source_

In [9]:
import os

# Example file path
file_path = results[0]['identity'][0]

# Extracting the name without the extension
file_name = os.path.splitext(os.path.basename(file_path))[0]

print(file_name)

Renjith Anil


In [10]:
import numpy as np

In [11]:
#xlwings
#initializing workbook and worksheet
workbook = xw.Book('attendance.xlsx')
sheet_name = datetime.date.today().isoformat()

try:
    worksheet = workbook.sheets(sheet_name)
except:
    worksheet = workbook.sheets.add(sheet_name)

In [12]:
#xlwings

worksheet.range('A1').value = 'STUDENT'
worksheet.range('B1').value = 'DATE'
worksheet.range('C1').value = 'TIME'

students=[]

In [13]:
def add_face(frame, name):
    # Save the new face image in the attendance folder
    path = f"./ImagesAttendance/{name}.jpg"
    cv2.imwrite(path, frame)
    print(f"Face added for {name}")
    return path

In [14]:
def save_cropped_face(frame, xmin, ymin, xmax, ymax, name):
    # Crop the face region
    face_crop = frame[ymin:ymax, xmin:xmax]
    
    # Save the cropped face image
    path = f"./ImagesAttendance/{name}.jpg"
    cv2.imwrite(path, face_crop)
    print(f"Face saved for {name}")
    return path

# Gesture Detection

In [15]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [16]:
folder = './HandGesture/images'

In [17]:
gesture_labels = {
    'call_me': 0,
    'fingers_crossed': 1,
    'okay': 2,
    'paper': 3,
    'peace': 4,
    'rock': 5,
    'rock_on': 6,
    'scissor': 7,
    'thumbs': 8,
    'up': 9
}

In [18]:
!C:\Users\renji\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [19]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(60, 60, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

In [20]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [21]:
def load_data(folder):
    images = []
    labels = []
    
    for gesture_dir in os.listdir(folder):
        label = gesture_labels.get(gesture_dir)
        if label is None:
            continue  
        path = os.path.join(folder, gesture_dir)
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (60, 60))  
            images.append(img)
            labels.append(label)
    
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

In [22]:
images, labels = load_data(folder)
x_train, x_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

In [23]:
train_gen = ImageDataGenerator(rescale=1.0/255.0)
val_gen = ImageDataGenerator(rescale=1.0/255.0)

In [24]:
model.fit(train_gen.flow(x_train, y_train, batch_size=32), validation_data=val_gen.flow(x_val, y_val, batch_size=32), epochs=10)

Epoch 1/10
132/132 [==============================] - 11s 70ms/step - loss: 1.0897 - accuracy: 0.6216 - val_loss: 0.3402 - val_accuracy: 0.8951
Epoch 2/10
132/132 [==============================] - 9s 68ms/step - loss: 0.2527 - accuracy: 0.9194 - val_loss: 0.3206 - val_accuracy: 0.8913
Epoch 3/10
132/132 [==============================] - 9s 66ms/step - loss: 0.1050 - accuracy: 0.9664 - val_loss: 0.1735 - val_accuracy: 0.9485
Epoch 4/10
132/132 [==============================] - 9s 67ms/step - loss: 0.0570 - accuracy: 0.9812 - val_loss: 0.1699 - val_accuracy: 0.9428
Epoch 5/10
132/132 [==============================] - 9s 66ms/step - loss: 0.0274 - accuracy: 0.9921 - val_loss: 0.1723 - val_accuracy: 0.9495
Epoch 6/10
132/132 [==============================] - 9s 66ms/step - loss: 0.0143 - accuracy: 0.9959 - val_loss: 0.0859 - val_accuracy: 0.9752
Epoch 7/10
132/132 [==============================] - 9s 67ms/step - loss: 0.0054 - accuracy: 0.9981 - val_loss: 0.0903 - val_accuracy: 0.976

# Functions for Gesture Detection


In [25]:
# Initialize gesture recognition model and MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

In [26]:
# Show a random gesture for the user to match
def show_random():
    gesture_dir = random.choice(list(gesture_labels.keys()))
    gesture_path = os.path.join(folder, gesture_dir)
    random_image_path = os.path.join(gesture_path, random.choice(os.listdir(gesture_path)))
    img = cv2.imread(random_image_path)
    cv2.imshow('Gesture to Match', img)
    cv2.waitKey(3000)
    cv2.destroyWindow('Gesture to Match')
    gesture_label = gesture_labels[gesture_dir]
    return gesture_label

In [27]:
# Detect hand and draw landmarks
def detect(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img_rgb)

    if result.multi_hand_landmarks:
        for hand in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(img, hand, mp_hands.HAND_CONNECTIONS)
    return img, result.multi_hand_landmarks


In [28]:
# Predict gesture
def predict(roi):
    if roi.size == 0:
        return None

    roi_resized = cv2.resize(roi, (60, 60))
    roi_resized = np.expand_dims(roi_resized, axis=0) / 255.0
    pred = model.predict(roi_resized)[0]

    ind = np.argsort(pred)[-5:][::-1]
    prob = pred[ind]
    labels = [(i, p) for i, p in zip(ind, prob)]
    return ind

In [29]:
def check(pred, true):
    if true in pred:
        print("Match found!")
        return True
    else:
        print("No match found. Try again.")
        return False

In [30]:
model2 = 'liveness.model'
model2 = tf.keras.models.load_model(model2)

In [33]:
# Initialize variables
cap = cv2.VideoCapture(0)
running = True
students = []
t0 = datetime.datetime.now().day  # Set t0 to the current day at the start
s = 2  # Row where the student details will be saved

while running:
    moment = datetime.datetime.now()
    hour = moment.hour
    minute = moment.minute
    day = moment.day
    month = moment.month
    year = moment.year
    date = f"{day}-{month}-{year}"
    time = f"{hour}:{minute}"

    # Check if the day has changed
    if day != t0:
        t0 = day
        worksheet = workbook.sheets.add(date)
        worksheet.range('A1').value = 'STUDENT'
        worksheet.range('B1').value = 'DATE'
        worksheet.range('C1').value = 'TIME'
        s = 2

    state, frame = cap.read()
    if not state:
        break

    # Perform face recognition
    res = DeepFace.find(frame, db_path='./ImagesAttendance/', enforce_detection=False, model_name='VGG-Face')

    if len(res[0]['identity']) > 0:
        name = res[0]['identity'][0].split('/')[2].split('\\')[0]
        xmin = int(res[0]['source_x'][0])
        ymin = int(res[0]['source_y'][0])
        w = res[0]['source_w'][0]
        h = res[0]['source_h'][0]
        xmax = int(xmin + w)
        ymax = int(ymin + h)

        img = frame[ymin:ymax, xmin:xmax]
        img = cv2.resize(img, (32, 32)).astype('float') / 255.0
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        liveness = model2.predict(img)
        liveness_value = liveness[0].argmax()

        # Draw bounding box and display name
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        cv2.putText(frame, name, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        liveness_text = f"Liveness: {'Real' if liveness_value == 1 else 'Fake'}"
        cv2.putText(frame, liveness_text, (xmin, ymax + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        if name in students:
            existing_date = worksheet.range(f'B{students.index(name) + 2}').value
            existing_time = worksheet.range(f'C{students.index(name) + 2}').value
            cv2.putText(frame, f"Logged in at {existing_time} on {existing_date}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        
        elif liveness_value == 1:
            cap.release()  # Release initial capture for gesture verification
            cv2.destroyAllWindows()

            # Start gesture recognition after closing the liveness window
            cap = cv2.VideoCapture(0)
            label = show_random()
            attempts = 0

            while attempts < 5:
                ret, gesture_frame = cap.read()
                processed_frame, hand = detect(gesture_frame)

                if hand:
                    h, w, _ = processed_frame.shape
                    x_min, y_min = w, h
                    x_max, y_max = 0, 0

                    for lm in hand[0].landmark:
                        x, y = int(lm.x * w), int(lm.y * h)
                        x_min, y_min = min(x, x_min), min(y, y_min)
                        x_max, y_max = max(x, x_max), max(y, y_max)

                    x_min, y_min = max(0, x_min - 20), max(0, y_min - 20)
                    x_max, y_max = min(w, x_max + 20), min(h, y_max + 20)

                    roi = processed_frame[y_min:y_max, x_min:x_max]
                    pred = predict(roi)

                    if check(pred, label):
                        cv2.putText(processed_frame, "Match!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                        print("Gesture matched!")
                        worksheet.range(f'A{s}').value, worksheet.range(f'B{s}').value, worksheet.range(f'C{s}').value = name, date, time
                        students.append(name)
                        s += 1
                        break
                    else:
                        cv2.putText(processed_frame, "Try again", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                        attempts += 1
                else:
                    cv2.putText(processed_frame, "No hand detected", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

                cv2.imshow('Gesture Recognition', processed_frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            cap.release()
            cv2.destroyAllWindows()

            # Reopen original video capture for liveness detection window
            cap = cv2.VideoCapture(0)
    else:
        cv2.putText(frame, "No face detected", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    cv2.imshow('Attendance System', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

24-11-07 01:54:05 - Searching [[[ 73  69 105]
  [ 71  66 108]
  [ 69  64 116]
  ...
  [139 128 138]
  [134 125 133]
  [129 121 128]]

 [[ 71  67  99]
  [ 69  66 103]
  [ 67  63 109]
  ...
  [138 126 136]
  [135 123 132]
  [131 121 128]]

 [[ 70  66  91]
  [ 68  65  93]
  [ 65  63  99]
  ...
  [141 125 135]
  [140 125 134]
  [138 123 131]]

 ...

 [[188 181 164]
  [189 183 165]
  [190 184 165]
  ...
  [255 255 237]
  [255 255 237]
  [255 255 237]]

 [[187 179 162]
  [187 180 162]
  [188 181 161]
  ...
  [255 255 236]
  [255 255 236]
  [255 255 236]]

 [[182 174 157]
  [183 176 158]
  [184 177 157]
  ...
  [255 255 236]
  [255 255 236]
  [255 255 236]]] in 10 length datastore
24-11-07 01:54:06 - find function duration 0.7092711925506592 seconds
24-11-07 01:54:06 - Searching [[[ 65  64  74]
  [ 65  64  74]
  [ 63  63  74]
  ...
  [ 96  97  98]
  [ 93  96  97]
  [ 92  95  96]]

 [[ 64  63  73]
  [ 63  63  73]
  [ 63  63  74]
  ...
  [ 97  97  98]
  [ 94  96  97]
  [ 92  95  96]]

 [[ 62  6

In [ ]:
# # Initialize variables
# cap = cv2.VideoCapture(0)
# running = True
# students = []

# t0 = datetime.datetime.now().day  # Set t0 to the current day at the start
# s = 2  # Row where the student details will be saved

# while running:
#     moment = datetime.datetime.now()
#     hour = moment.hour
#     minute = moment.minute
#     day = moment.day
#     month = moment.month
#     year = moment.year

#     date = f"{day}-{month}-{year}"
#     time = f"{hour}:{minute}"

#     # Check if the day has changed
#     if day != t0:
#         t0 = day
#         worksheet = workbook.sheets.add(date)
#         worksheet.range('A1').value = 'STUDENT'
#         worksheet.range('B1').value = 'DATE'
#         worksheet.range('C1').value = 'TIME'
#         s = 2

#     state, frame = cap.read()
#     if not state:
#         break

#     # Perform face recognition
#     res = DeepFace.find(frame, db_path='./ImagesAttendance/', enforce_detection=False, model_name='VGG-Face')

#     if len(res[0]['identity']) > 0:
#         # Extracting the name without the extension
#         name = res[0]['identity'][0].split('/')[2].split('\\')[0]

#         # Get the face coordinates
#         xmin = int(res[0]['source_x'][0])
#         ymin = int(res[0]['source_y'][0])
#         w = res[0]['source_w'][0]
#         h = res[0]['source_h'][0]
#         xmax = int(xmin + w)
#         ymax = int(ymin + h)

#         # Liveness detection
#         img = frame[ymin:ymax, xmin:xmax]
#         img = cv2.resize(img, (32, 32))
#         img = img.astype('float') / 255.0
#         img = tf.keras.preprocessing.image.img_to_array(img)
#         img = np.expand_dims(img, axis=0)

#         liveness = model.predict(img)
#         liveness_value = liveness[0].argmax()  # 0 for fake, 1 for real

#         # Draw the bounding box and display name
#         cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
#         cv2.rectangle(frame, (xmin, ymin - 25), (xmax, ymin), (255, 255, 255), -1)
#         cv2.putText(frame, name, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2, cv2.LINE_AA)

#         # Display liveness result
#         liveness_text = f"Liveness: {'Real' if liveness_value == 1 else 'Fake'}"
#         cv2.putText(frame, liveness_text, (xmin, ymax + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

#         # Display date and time on the frame
#         cv2.putText(frame, f"Date: {date}", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
#         cv2.putText(frame, f"Time: {time}", (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

#         # Log attendance if liveness is real and name is not already logged
#         if liveness_value == 1 and name not in students:
#             worksheet.range(f'A{s}').value = name
#             worksheet.range(f'B{s}').value = date
#             worksheet.range(f'C{s}').value = time
#             students.append(name)
#             s += 1
#     else:
#         cv2.putText(frame, "No face detected", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)

#     # Show the frame with OpenCV
#     cv2.imshow('Attendance System', frame)

#     # Check for the exit condition (press 'q' to exit)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()


find function lasts  0.17710089683532715  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18119215965270996  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18382835388183594  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1781752109527588  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.19152069091796875  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1807868480682373  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  repre

find function lasts  0.17470622062683105  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.17293667793273926  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.16732168197631836  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.17200922966003418  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.16824603080749512  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1723928451538086  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  repr

find function lasts  0.16996121406555176  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1796250343322754  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.17474699020385742  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.17446160316467285  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.17377376556396484  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18857526779174805  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  repr

find function lasts  0.17311859130859375  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18978190422058105  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.17669296264648438  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1712329387664795  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1851499080657959  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1941361427307129  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  repres

find function lasts  0.1855151653289795  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.19018888473510742  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1726217269897461  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18089008331298828  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18118596076965332  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1854259967803955  seconds
1/1 [==============================] - 0s 12ms/step
There are  9  representations found in  repres

...

find function lasts  0.18538880348205566  seconds
1/1 [==============================] - 0s 15ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1843397617340088  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18799352645874023  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18326759338378906  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18192243576049805  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1871805191040039  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  repre

find function lasts  0.18716883659362793  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1879291534423828  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18351459503173828  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18503165245056152  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18235516548156738  seconds
1/1 [==============================] - 0s 15ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18480491638183594  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  repr

find function lasts  0.1870250701904297  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1865220069885254  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18695688247680664  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18448758125305176  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1862494945526123  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.19168663024902344  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  repres

find function lasts  0.18854427337646484  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18583369255065918  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18427395820617676  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18417954444885254  seconds
1/1 [==============================] - 0s 15ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.18796348571777344  seconds
1/1 [==============================] - 0s 13ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1851356029510498  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  repr

find function lasts  0.19412684440612793  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1967165470123291  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.19788026809692383  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.1928846836090088  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.19351840019226074  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  representations_vgg_face.pkl
find function lasts  0.19916367530822754  seconds
1/1 [==============================] - 0s 14ms/step
There are  9  representations found in  repre

In [ ]:
import tensorflow as tf
print(tf.__version__)
